# UV-Vis Layer Stack Simulator
**Transfer Matrix Method (TMM) Calculator for Optical Properties**

Based on the MATLAB code by Malte Langenhorst et al. (KIT)

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries loaded")

✓ Libraries loaded


## Core Functions

In [2]:
# type: ignore
# Helper functions for degree-based trigonometry (like MATLAB)
def sind(theta_deg):
    """sin in degrees (like MATLAB sind)"""
    return np.sin(np.asarray(theta_deg) * np.pi / 180.0)

def cosd(theta_deg):
    """cos in degrees (like MATLAB cosd)"""
    return np.cos(np.asarray(theta_deg) * np.pi / 180.0)

def tand(theta_deg):
    """tan in degrees (like MATLAB tand)"""
    return np.tan(np.asarray(theta_deg) * np.pi / 180.0)

def arcsind(x):
    """arcsin in degrees (like MATLAB asind)"""
    return np.arcsin(np.asarray(x)) * 180.0 / np.pi

def arccosd(x):
    """arccos in degrees (like MATLAB acosd)"""
    return np.arccos(np.asarray(x)) * 180.0 / np.pi


def is_forward_angle(n, theta):
    """Check if a wave traveling at angle theta in medium with index n is forward-traveling."""
    if np.max(np.real(n) * np.imag(n) < 0):
        raise ValueError("For materials with gain, it's ambiguous which beam is incoming vs outgoing.")
    
    if np.isscalar(theta):
        theta = np.ones_like(n) * theta
    
    ncostheta = n * cosd(theta)
    answer = np.zeros(n.shape, dtype=bool)
    
    evanescent = np.abs(np.imag(ncostheta)) > 100 * np.finfo(float).eps
    answer[evanescent] = np.imag(ncostheta[evanescent]) > 0
    
    propagating = ~evanescent
    answer[propagating] = np.real(ncostheta[propagating]) > 0
    
    return answer


def interface_r(pol, n_i, n_f, th_i, th_f):
    """Reflection amplitude at interface between two media."""
    if pol == 'TE':
        r = ((n_i * cosd(th_i) - n_f * cosd(th_f)) /
             (n_i * cosd(th_i) + n_f * cosd(th_f)))
    elif pol == 'TM':
        r = ((n_f * cosd(th_i) - n_i * cosd(th_f)) /
             (n_f * cosd(th_i) + n_i * cosd(th_f)))
    else:
        raise ValueError("Polarization must be 'TE' or 'TM'")
    return r


def interface_t(pol, n_i, n_f, th_i, th_f):
    """Transmission amplitude at interface between two media."""
    if pol == 'TE':
        t = (2 * n_i * cosd(th_i) /
             (n_i * cosd(th_i) + n_f * cosd(th_f)))
    elif pol == 'TM':
        t = (2 * n_i * cosd(th_i) /
             (n_f * cosd(th_i) + n_i * cosd(th_f)))
    else:
        raise ValueError("Polarization must be 'TE' or 'TM'")
    return t


def R_from_r(r):
    """Calculate reflectance from reflection amplitude."""
    return np.abs(r * r)


def T_from_t(pol, t, n_i, n_f, th_i, th_f):
    """Calculate transmittance from transmission amplitude."""
    if pol == 'TE':
        T = (np.abs(t * t) * 
             (np.real(n_f * cosd(th_f)) / 
              np.real(n_i * cosd(th_i))))
    elif pol == 'TM':
        T = (np.abs(t * t) * 
             (np.real(n_f * np.conj(cosd(th_f))) / 
              np.real(n_i * np.conj(cosd(th_i)))))
    else:
        raise ValueError("Polarization must be 'TE' or 'TM'")
    return T


def power_entering_from_r(pol, r, n_i, th_i):
    """Calculate power entering from reflection amplitude."""
    if pol == 'TE':
        P_in = (np.real(n_i * cosd(th_i) * (1 + np.conj(r)) * (1 - r)) /
                np.real(n_i * cosd(th_i)))
    elif pol == 'TM':
        P_in = (np.real(n_i * np.conj(cosd(th_i)) * (1 + r) * (1 - np.conj(r))) /
                np.real(n_i * np.conj(cosd(th_i))))
    else:
        raise ValueError("Polarization must be 'TE' or 'TM'")
    return P_in


def list_snell(n_list, th_0):
    """Apply Snell's law to get angle in each layer.
    n0 * sin(th0) = ni * sin(thi)
    => sin(thi) = n0/ni * sin(th0)
    => thi = arcsin(n0/ni * sin(th0))
    """
    num_lambdas, num_layers = n_list.shape
    angles = np.zeros((num_lambdas, num_layers), dtype=complex)
    
    # Direct Snell's law calculation for all layers
    sin_th_0 = n_list[:, 0] * sind(th_0)
    for i in range(num_layers):
        angles[:, i] = arcsind(sin_th_0 / n_list[:, i])
    
    # Correct first and last layer angles to ensure forward-traveling wave
    # First layer
    idx_first = is_forward_angle(n_list[:, 0], angles[:, 0])
    angles[~idx_first, 0] = 90 - angles[~idx_first, 0]
    
    # Last layer
    idx_last = is_forward_angle(n_list[:, -1], angles[:, -1])
    angles[~idx_last, -1] = 90 - angles[~idx_last, -1]
    
    return angles

In [3]:
# type: ignore
def coh_tmm(pol, n_array, d_list, th_0, lam_vac):
    """
    Main coherent transfer matrix method calculation.
    Exact translation of MATLAB coh_tmm.m algorithm.
    """
    if np.ndim(th_0) > 0 and len(th_0) > 1:
        raise ValueError('This function is not vectorized for multiple angles')
    
    num_layers = len(d_list)
    num_lambdas = len(lam_vac)
    
    if d_list[0] != np.inf or d_list[-1] != np.inf:
        raise ValueError('d_list must start and end with inf!')
    
    n_array[:, 0] = np.real(n_array[:, 0])
    
    # Apply Snell's law to get angles in each layer
    th_array = list_snell(n_array, th_0)
    
    # Calculate kz: normal component of angular wavenumber
    # kz_array has shape (num_lambdas, num_layers)
    kz_array = 2 * np.pi * n_array * cosd(th_array) / lam_vac[:, np.newaxis]
    
    # Calculate phase accumulation delta = kz * d
    delta = kz_array * d_list
    
    # For very opaque layers (imag(delta) > 35), set imag to 35 while preserving real part
    for i in range(1, num_layers - 1):
        mask = np.imag(delta[:, i]) > 35
        delta[mask, i] = np.real(delta[mask, i]) + 35j
    
    # Calculate Fresnel coefficients at all interfaces
    t_list = np.zeros((num_layers, num_layers, num_lambdas), dtype=complex)
    r_list = np.zeros((num_layers, num_layers, num_lambdas), dtype=complex)
    
    for i in range(num_layers - 1):
        t_list[i, i+1, :] = interface_t(pol, n_array[:, i], n_array[:, i+1], 
                                        th_array[:, i], th_array[:, i+1])
        r_list[i, i+1, :] = interface_r(pol, n_array[:, i], n_array[:, i+1],
                                        th_array[:, i], th_array[:, i+1])
    
    # Build M_list: (num_layers, 2, 2, num_lambdas)
    # M_list[i] is the characteristic matrix for layer i
    M_list = np.zeros((num_layers, 2, 2, num_lambdas), dtype=complex)
    
    for i in range(1, num_layers - 1):
        # M_list(i,1,1,:) = (1/t_list(i,i+1,:)) * exp(-1j*delta(:,i))
        M_list[i, 0, 0, :] = (1 / t_list[i, i+1, :]) * np.exp(-1j * delta[:, i])
        
        # M_list(i,1,2,:) = (r_list(i,i+1,:)/t_list(i,i+1,:)) * exp(-1j*delta(:,i))
        M_list[i, 0, 1, :] = (r_list[i, i+1, :] / t_list[i, i+1, :]) * np.exp(-1j * delta[:, i])
        
        # M_list(i,2,1,:) = (r_list(i,i+1,:)/t_list(i,i+1,:)) * exp(1j*delta(:,i))
        M_list[i, 1, 0, :] = (r_list[i, i+1, :] / t_list[i, i+1, :]) * np.exp(1j * delta[:, i])
        
        # M_list(i,2,2,:) = (1/t_list(i,i+1,:)) * exp(1j*delta(:,i))
        M_list[i, 1, 1, :] = (1 / t_list[i, i+1, :]) * np.exp(1j * delta[:, i])
    
    # Build Mtilde: accumulated scattering matrix (2, 2, num_lambdas)
    # Start with identity matrix for each wavelength
    Mtilde = np.zeros((2, 2, num_lambdas), dtype=complex)
    Mtilde[0, 0, :] = 1.0
    Mtilde[1, 1, :] = 1.0
    
    # Forward loop: multiply Mtilde by M_list for layers 2 to num_layers-1
    for i in range(1, num_layers - 1):
        Mtilde_temp = Mtilde.copy()
        
        # Mtilde = Mtilde @ M_list[i] (matrix multiplication for each wavelength)
        Mtilde[0, 0, :] = (Mtilde_temp[0, 0, :] * M_list[i, 0, 0, :] + 
                           Mtilde_temp[0, 1, :] * M_list[i, 1, 0, :])
        Mtilde[0, 1, :] = (Mtilde_temp[0, 0, :] * M_list[i, 0, 1, :] + 
                           Mtilde_temp[0, 1, :] * M_list[i, 1, 1, :])
        Mtilde[1, 0, :] = (Mtilde_temp[1, 0, :] * M_list[i, 0, 0, :] + 
                           Mtilde_temp[1, 1, :] * M_list[i, 1, 0, :])
        Mtilde[1, 1, :] = (Mtilde_temp[1, 0, :] * M_list[i, 0, 1, :] + 
                           Mtilde_temp[1, 1, :] * M_list[i, 1, 1, :])
    
    # Apply layer 1 interface to Mtilde
    Mtilde_temp = Mtilde.copy()
    Mtilde[0, 0, :] = (1 / t_list[0, 1, :]) * Mtilde_temp[0, 0, :] + (r_list[0, 1, :] / t_list[0, 1, :]) * Mtilde_temp[1, 0, :]
    Mtilde[0, 1, :] = (1 / t_list[0, 1, :]) * Mtilde_temp[0, 1, :] + (r_list[0, 1, :] / t_list[0, 1, :]) * Mtilde_temp[1, 1, :]
    Mtilde[1, 0, :] = (r_list[0, 1, :] / t_list[0, 1, :]) * Mtilde_temp[0, 0, :] + (1 / t_list[0, 1, :]) * Mtilde_temp[1, 0, :]
    Mtilde[1, 1, :] = (r_list[0, 1, :] / t_list[0, 1, :]) * Mtilde_temp[0, 1, :] + (1 / t_list[0, 1, :]) * Mtilde_temp[1, 1, :]
    
    # Extract overall r and t from Mtilde
    r = Mtilde[1, 0, :] / Mtilde[0, 0, :]
    t = 1.0 / Mtilde[0, 0, :]
    
    # Calculate reflectance and transmittance
    R = R_from_r(r)
    T = T_from_t(pol, t, n_array[:, 0], n_array[:, -1], 
                 th_array[:, 0], th_array[:, -1])
    
    power_entering = power_entering_from_r(pol, r, n_array[:, 0], th_array[:, 0])
    
    # Build vw_array: forward/backward amplitudes at each interface
    # vw_array[n] = [v_n, w_n]
    vw_array = np.zeros((num_layers, 2, num_lambdas), dtype=complex)
    
    # Start at the last layer with transmission amplitude
    vw = np.zeros((2, num_lambdas), dtype=complex)
    vw[0, :] = t
    vw[1, :] = 0
    vw_array[-1, :, :] = vw
    
    # Backward recursion: for i from num_layers-2 down to 1
    for i in range(num_layers - 2, 0, -1):
        vw_temp = vw.copy()
        
        # vw = M_list[i] @ vw
        vw[0, :] = M_list[i, 0, 0, :] * vw_temp[0, :] + M_list[i, 0, 1, :] * vw_temp[1, :]
        vw[1, :] = M_list[i, 1, 0, :] * vw_temp[0, :] + M_list[i, 1, 1, :] * vw_temp[1, :]
        
        vw_array[i, :, :] = vw
    
    return {
        'r': r, 't': t, 'R': R, 'T': T, 'power_entering': power_entering,
        'vw_array': vw_array, 'kz_array': kz_array, 'th_array': th_array,
        'pol': pol, 'n_array': n_array, 'd_list': d_list, 'th_0': th_0, 'lam_vac': lam_vac
    }

In [4]:
# type: ignore
def find_in_structure_with_inf(d_list, dist):
    """Find which layer a given distance corresponds to."""
    if dist < 0:
        return 1, dist
    cumsum = 0
    for i in range(1, len(d_list) - 1):
        if cumsum <= dist < cumsum + d_list[i]:
            return i, dist - cumsum
        cumsum += d_list[i]
    return len(d_list) - 1, dist - cumsum


def position_resolved(layer, distance, coh_tmm_data):
    """Calculate Poynting vector and absorption at specific position."""
    num_lambdas = len(coh_tmm_data['lam_vac'])
    
    if layer > 0:
        v = coh_tmm_data['vw_array'][layer, 0, :]
        w = coh_tmm_data['vw_array'][layer, 1, :]
    else:
        v = np.ones(num_lambdas)
        w = coh_tmm_data['r']
    
    kz = coh_tmm_data['kz_array'][:, layer]
    th = coh_tmm_data['th_array'][:, layer]
    n = coh_tmm_data['n_array'][:, layer]
    n_0 = coh_tmm_data['n_array'][:, 0]
    th_0 = coh_tmm_data['th_0']
    pol = coh_tmm_data['pol']
    
    Ef = v * np.exp(1j * kz * distance)
    Eb = w * np.exp(-1j * kz * distance)
    
    if pol == 'TE':
        poyn = (np.real(n * cosd(th) * np.conj(Ef + Eb) * (Ef - Eb)) /
                np.real(n_0 * cosd(th_0)))
    elif pol == 'TM':
        poyn = (np.real(n * np.conj(cosd(th)) * (Ef + Eb) * np.conj(Ef - Eb)) /
                np.real(n_0 * np.conj(cosd(th_0))))
    
    if pol == 'TE':
        absor = (np.imag(n * cosd(th) * kz) * np.abs(Ef + Eb)**2 /
                np.real(n_0 * cosd(th_0)))
    elif pol == 'TM':
        absor = (np.imag(n * np.conj(cosd(th)) * 
                (kz * np.abs(Ef - Eb)**2 - np.conj(kz) * np.abs(Ef + Eb)**2)) /
                np.real(n_0 * np.conj(cosd(th_0))))
    
    if pol == 'TE':
        Ex = np.zeros(num_lambdas)
        Ey = Ef + Eb
        Ez = np.zeros(num_lambdas)
    elif pol == 'TM':
        Ex = (Ef - Eb) * cosd(th)
        Ey = np.zeros(num_lambdas)
        Ez = (-Ef - Eb) * sind(th)
    
    return {'poyn': poyn, 'absor': absor, 'Ex': Ex, 'Ey': Ey, 'Ez': Ez}


def absorp_in_each_layer(coh_tmm_data):
    """Calculate absorption in each layer."""
    num_lambdas = len(coh_tmm_data['lam_vac'])
    num_layers = len(coh_tmm_data['d_list'])
    
    power_entering_each_layer = np.zeros((num_lambdas, num_layers))
    power_entering_each_layer[:, 0] = 1.0
    power_entering_each_layer[:, 1] = coh_tmm_data['power_entering']
    power_entering_each_layer[:, -1] = coh_tmm_data['T']
    
    for i in range(2, num_layers - 1):
        data = position_resolved(i, 0, coh_tmm_data)
        power_entering_each_layer[:, i] = data['poyn']
    
    final_answer = np.zeros((num_lambdas, num_layers))
    final_answer[:, :-1] = -np.diff(power_entering_each_layer, axis=1)
    final_answer[:, -1] = power_entering_each_layer[:, -1]
    final_answer[final_answer < np.finfo(float).eps] = 0
    
    return final_answer

In [5]:
# type: ignore
class RefractiveIndexLibrary:
    """Manage refractive index data for materials."""
    
    def __init__(self):
        self.data = None
        self.material_names = []
        self.loaded = False
        
    def load_from_excel(self, filepath):
        """Load refractive index data from Excel file."""
        try:
            df = pd.read_excel(filepath)
            self.data = df
            
            self.material_names = []
            for col in df.columns:
                if col.endswith('_n'):
                    mat_name = col[:-2]
                    if f'{mat_name}_k' in df.columns:
                        self.material_names.append(mat_name)
            
            # Sort materials alphabetically
            self.material_names.sort()
            self.loaded = True
            return True
        except Exception as e:
            print(f"Error: {e}")
            return False
    
    def load_predefined(self):
        """Load predefined materials for testing."""
        wavelengths = np.arange(300, 1000, 1)
        data = {
            'Wavelength (nm)': wavelengths,
            'Air_n': np.ones_like(wavelengths, dtype=float),
            'Air_k': np.zeros_like(wavelengths, dtype=float),
            'Glass_n': np.ones_like(wavelengths, dtype=float) * 1.5,
            'Glass_k': np.zeros_like(wavelengths, dtype=float),
            'ITO_n': np.ones_like(wavelengths, dtype=float) * 2.0,
            'ITO_k': np.ones_like(wavelengths, dtype=float) * 0.01,
        }
        self.data = pd.DataFrame(data)
        self.material_names = sorted(['Air', 'Glass', 'ITO'])
        self.loaded = True
        return True
    
    def get_refractive_index(self, material_name, wavelengths):
        """Get complex refractive index for a material at given wavelengths."""
        if self.data is None:
            raise ValueError("No refractive index data loaded!")
        
        if material_name not in self.material_names:
            raise ValueError(f"Material '{material_name}' not found!")
        
        wl_data = self.data['Wavelength (nm)'].values
        n_data = self.data[f'{material_name}_n'].values
        k_data = self.data[f'{material_name}_k'].values
        
        n_interp = np.interp(wavelengths, wl_data, n_data)
        k_interp = np.interp(wavelengths, wl_data, k_data)
        
        return n_interp + 1j * k_interp
    
    def get_available_materials(self):
        """Return list of available materials (sorted)."""
        return sorted(self.material_names.copy())


# Initialize the library
refr_index_lib = RefractiveIndexLibrary()

# Try to load Excel file, fall back to predefined
excel_file_path = 'Index_of_Refraction_library2.xls'
if not refr_index_lib.load_from_excel(excel_file_path):
    refr_index_lib.load_predefined()

status_msg = "✓ Index loaded from Excel" if refr_index_lib.loaded else "✓ Using predefined materials"
print(status_msg)

✓ Index loaded from Excel


In [ ]:
# type: ignore
class LayerStackSimulator:
    """Interactive simulator for multilayer optical stacks."""
    
    def __init__(self, refr_index_lib):
        self.refr_index_lib = refr_index_lib
        self.layers = []
        self.layer_widgets = []
        self.previous_selections = {}  # Remember user selections
        self.create_widgets()
        
    def create_widgets(self):
        """Create interactive widgets."""
        style = {'description_width': '150px'}
        
        # Simulation parameters
        self.wl_min = widgets.IntText(value=400, description='Min λ (nm):', style=style)
        self.wl_max = widgets.IntText(value=900, description='Max λ (nm):', style=style)
        self.angle_inc = widgets.FloatSlider(value=0, min=0, max=85, step=1,
                                            description='Angle (°):', style=style)
        self.polarization = widgets.Dropdown(options=['TE', 'TM'], value='TE',
                                            description='Polarization:', style=style)
        self.glass_substrate = widgets.Checkbox(value=True, 
                                               description='Glass substrate correction',
                                               style={'description_width': 'initial'})
        
        # Layer configuration
        self.num_layers = widgets.IntText(value=4, description='Number of layers:', 
                                         style=style, min=2, max=10)
        
        self.layer_container = widgets.VBox([])
        
        # Buttons
        self.update_layers_btn = widgets.Button(description='Update Layer Configuration',
                                               button_style='info', tooltip='Change number of layers')
        self.calculate_btn = widgets.Button(description='Calculate', button_style='success')
        self.help_btn = widgets.Button(description='Help', button_style='warning')
        
        # Help output
        self.help_output = widgets.Output()
        self.help_visible = False
        
        # Calculation output
        self.output = widgets.Output()
        
        # Connect events
        self.update_layers_btn.on_click(self.update_layer_widgets)
        self.calculate_btn.on_click(self.calculate_and_plot)
        self.help_btn.on_click(self.toggle_help)
        
        # Initial layer setup
        self.update_layer_widgets(None)
        
    def toggle_help(self, b):
        """Toggle help panel visibility."""
        self.help_visible = not self.help_visible
        self.help_output.clear_output()
        
        if self.help_visible:
            with self.help_output:
                help_text = """
<h4>How to Use This Simulator</h4>
<b>Parameters:</b>
<ul>
<li><b>Min/Max λ:</b> Wavelength range in nm (300-1000 nm typical)</li>
<li><b>Angle:</b> Incident angle in degrees (0° = normal incidence)</li>
<li><b>Polarization:</b> TE (s-polarized) or TM (p-polarized)</li>
<li><b>Glass correction:</b> Adjust for substrate effects (R+5%, T×95%)</li>
</ul>

<b>Stack Configuration:</b>
<ul>
<li>First and last layers are semi-infinite (substrate/superstrate)</li>
<li>Adjust number of layers and click "Update"</li>
<li>Select material and thickness for each layer</li>
<li>Click "Calculate" to run the simulation</li>
</ul>

<b>Output:</b>
<ul>
<li><b>R, T, A:</b> Reflectance, Transmittance, Absorptance</li>
<li><b>Absorbance:</b> Optical density (log scale)</li>
<li><b>Layer Absorption:</b> Absorption in each layer</li>
</ul>

<b>Example Stack (Perovskite Solar Cell):</b>
<br>Glass (∞) → ITO (130 nm) → Perovskite (500 nm) → Air (∞)
                """
                display(HTML(help_text))
    
    def update_layer_widgets(self, b):
        """Update layer input widgets, preserving previous selections."""
        num = self.num_layers.value
        materials = self.refr_index_lib.get_available_materials()
        
        self.layer_widgets = []
        layer_boxes = []
        
        for i in range(num):
            # First and last layers are semi-infinite
            if i == 0 or i == num - 1:
                thickness = widgets.FloatText(value=np.inf, disabled=True,
                                             style={'description_width': '180px'})
            else:
                # Restore previous thickness if it exists
                prev_thickness = self.previous_selections.get(f'thickness_{i}', 100)
                thickness = widgets.FloatText(value=prev_thickness,
                                             style={'description_width': '180px'})
            
            # Restore previous material selection
            prev_material = self.previous_selections.get(f'material_{i}')
            if prev_material and prev_material in materials:
                default_mat = prev_material
            else:
                default_mat = materials[min(i, len(materials)-1)]
            
            material = widgets.Dropdown(options=materials, value=default_mat,
                                       style={'description_width': '180px'})
            
            # Add change listeners to save selections
            material.observe(lambda change, idx=i: self._save_selection('material', idx, change), names='value')
            thickness.observe(lambda change, idx=i: self._save_selection('thickness', idx, change), names='value')
            
            layer_box = widgets.HBox([material, thickness])
            layer_boxes.append(layer_box)
            self.layer_widgets.append({'material': material, 'thickness': thickness})
        
        self.layer_container.children = layer_boxes
    
    def _save_selection(self, param_type, index, change):
        """Save user selections for later restoration."""
        key = f'{param_type}_{index}'
        self.previous_selections[key] = change['new']
    
    def get_stack_configuration(self):
        """Extract current stack configuration."""
        stack_names = []
        thicknesses = []
        
        for layer in self.layer_widgets:
            stack_names.append(layer['material'].value)
            thicknesses.append(layer['thickness'].value)
        
        return stack_names, thicknesses
    
    def calculate_and_plot(self, b):
        """Run TMM calculation and plot results."""
        with self.output:
            clear_output(wait=True)
            
            try:
                stack_names, thicknesses = self.get_stack_configuration()
                lam_vac = np.arange(self.wl_min.value, self.wl_max.value + 1, 1)
                
                # Build refractive index array
                n_array = np.zeros((len(lam_vac), len(stack_names)), dtype=complex)
                for i, mat_name in enumerate(stack_names):
                    n_array[:, i] = self.refr_index_lib.get_refractive_index(mat_name, lam_vac)
                
                # Run TMM
                d_list = np.array(thicknesses)
                coh_tmm_data = coh_tmm(self.polarization.value, n_array, d_list,
                                      self.angle_inc.value, lam_vac)
                
                # Get absorption in each layer
                A_layers = absorp_in_each_layer(coh_tmm_data)
                A_layers = np.clip(A_layers, 0, None)
                
                # Extract R, T, A
                R = coh_tmm_data['R']
                T = coh_tmm_data['T']
                A_total = 1 - R - T
                
                # Apply glass substrate correction
                if self.glass_substrate.value:
                    R_corr = R + 0.05
                    T_corr = T * 0.95
                else:
                    R_corr = R
                    T_corr = T
                A_corr = 1 - R_corr - T_corr
                
                # Clip to physical bounds
                R_clip = np.clip(R_corr, 0, 1)
                T_clip = np.clip(T_corr, 0, 1)
                A_clip = np.clip(A_corr, 0, 1)
                
                # Calculate absorbance
                denominator = 100 - R_clip * 100
                denominator[denominator == 0] = 1e-10
                absorbance = -np.log10(T_clip * 100 / denominator)
                
                # Plot results
                self.plot_results(lam_vac, R_clip, T_clip, A_clip, 
                                absorbance, A_layers, stack_names)
                
            except Exception as e:
                print(f"❌ Error: {e}")
    
    def plot_results(self, lam_vac, R, T, A, absorbance, A_layers, stack_names):
        """Plot optical properties using Plotly."""
        
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Reflectance, Transmittance, Absorptance', '', 
                          'Absorbance', 'Layer Absorption'),
            specs=[[{'colspan': 2}, None], [{}, {}]],
            vertical_spacing=0.12, horizontal_spacing=0.1
        )
        
        # Plot R, T, A
        fig.add_trace(go.Scatter(x=lam_vac, y=R, name='R', line=dict(width=2, color='#1f77b4'), mode='lines'),
                     row=1, col=1)
        fig.add_trace(go.Scatter(x=lam_vac, y=T, name='T', line=dict(width=2, color='#ff7f0e'), mode='lines'),
                     row=1, col=1)
        fig.add_trace(go.Scatter(x=lam_vac, y=A, name='A', line=dict(width=2, color='#2ca02c'), mode='lines'),
                     row=1, col=1)
        
        # Plot Absorbance
        fig.add_trace(go.Scatter(x=lam_vac, y=absorbance, name='Absorbance',
                                line=dict(width=2, color='purple'), mode='lines', showlegend=False),
                     row=2, col=1)
        
        # Plot Layer Absorption
        colors = ['#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf', '#aec7e8']
        for i in range(1, len(stack_names) - 1):
            color = colors[(i-1) % len(colors)]
            fig.add_trace(go.Scatter(x=lam_vac, y=A_layers[:, i], name=f'{stack_names[i]}',
                                    line=dict(width=2, color=color), mode='lines'),
                         row=2, col=2)
        
        # Update axes
        fig.update_xaxes(title_text="Wavelength (nm)", row=1, col=1)
        fig.update_yaxes(title_text="R, T, A", range=[0, 1], row=1, col=1)
        fig.update_xaxes(title_text="Wavelength (nm)", row=2, col=1)
        fig.update_yaxes(title_text="Absorbance", row=2, col=1)
        fig.update_xaxes(title_text="Wavelength (nm)", row=2, col=2)
        ymax = np.max(A_layers[:, 1:-1]) if A_layers[:, 1:-1].size else 0
        fig.update_yaxes(title_text="Absorption", range=[0, max(1e-6, ymax)], row=2, col=2)
        
        fig.update_layout(height=700, showlegend=True, hovermode='x unified',
                         template='plotly_white', font=dict(size=11),
                         margin=dict(l=50, r=50, t=80, b=50))
        
        fig.show()
        
        # Print summary
        print(f"λ: {lam_vac[0]}–{lam_vac[-1]} nm | Angle: {self.angle_inc.value}° | Pol: {self.polarization.value}")
        print(f"Avg: R={np.mean(R):.3f} | T={np.mean(T):.3f} | A={np.mean(A):.3f}")
    
    def display(self):
        """Display the interactive interface."""
        
        button_box = widgets.HBox([self.calculate_btn, self.help_btn])
        
        config_box = widgets.VBox([
            widgets.HTML("<h3>Parameters</h3>"),
            widgets.HBox([self.wl_min, self.wl_max]),
            widgets.HBox([self.angle_inc, self.polarization]),
            self.glass_substrate,
            widgets.HTML("<h3>Layer Stack</h3>"),
            widgets.HBox([self.num_layers, self.update_layers_btn]),
            self.layer_container,
            button_box,
            self.help_output,
            self.output
        ])
        
        display(config_box)


# Create and display simulator
simulator = LayerStackSimulator(refr_index_lib)
simulator.display()